global variables and functions

In [ ]:
import numpy as np
import cv2 as cv

# all reusable methodes


def path(name):
    return "./images/{}.png".format(name)

def show(img):
    cv.imshow("img", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

def read(img):
    image_path = path(img)
    image = cv.imread(image_path)
    return image

def write(img, name):
    image_path = path(name)
    cv.imwrite(image_path, img)

def readShow(img):
    image = read(img)
    show(image)
    return image


def writeShow(img, name):
    write(img, name)
    show(img)




def neighbor(mat, i, j):
    arr = []
    arr.append(mat[i-1][j-1])
    arr.append(mat[i-1][j])
    arr.append(mat[i-1][j+1])

    arr.append(mat[i][j-1])
    arr.append(mat[i][j+1])

    arr.append(mat[i+1][j-1])
    arr.append(mat[i+1][j])
    arr.append(mat[i+1][j+1])
    return arr


# all reusalbe variables

im = read("image")

gim = cv.cvtColor(im, cv.COLOR_BGR2GRAY)

gray scale

In [ ]:
# B,G,R = img[i,j]
# img[i,j] = (0.2989 * B + 0.5870 * G + 0.1140 * R)

fixed threshold binarization

In [ ]:
ret, thresh1 = cv.threshold(gim, 127, 255, cv.THRESH_BINARY)
ret, thresh2 = cv.threshold(gim, 127, 255, cv.THRESH_BINARY_INV)
ret, thresh3 = cv.threshold(gim, 127, 255, cv.THRESH_TRUNC)
ret, thresh4 = cv.threshold(gim, 127, 255, cv.THRESH_TOZERO)
ret, thresh5 = cv.threshold(gim, 127, 255, cv.THRESH_TOZERO_INV)
titles = ['BINARY', 'BINARY_INV', 'TRUNC', 'TOZERO', 'TOZERO_INV']
images = [thresh1, thresh2, thresh3, thresh4, thresh5]
for i in range(5):
    cv.imwrite(path(titles[i]), images[i])


adaptive global threshold binarization

In [ ]:
#predefined otsu method

ret2, cv2_otsu = cv.threshold(gim, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)


In [ ]:
#otsu's implementation

def thr_otsu_impl(image, nbins=1):

    #validate grayscale
    if len(image.shape) == 1 or len(image.shape) > 2:
        print("must be a grayscale image.")
        return

    #validate multicolored
    if np.min(image) == np.max(image):
        print("the image must have multiple pixel intenseties")
        return

    all_colors = image.flatten()
    total_weight = len(all_colors)
    least_variance = -1
    least_variance_threshold = -1

    # create an array of all possible threshold values which we want to loop through
    color_thresholds = np.arange(
        np.min(image)+nbins, np.max(image)-nbins, nbins)

    # loop through the thresholds to find the one with the least within class variance
    for color_threshold in color_thresholds:
        bg_pixels = all_colors[all_colors < color_threshold]
        weight_bg = len(bg_pixels) / total_weight
        variance_bg = np.var(bg_pixels)

        fg_pixels = all_colors[all_colors >= color_threshold]
        weight_fg = len(fg_pixels) / total_weight
        variance_fg = np.var(fg_pixels)

        within_class_variance = weight_fg*variance_fg + weight_bg*variance_bg
        if least_variance == -1 or least_variance > within_class_variance:
            least_variance = within_class_variance
            least_variance_threshold = color_threshold

    return least_variance_threshold


thr = thr_otsu_impl(gim)
thresh, my_otsu = cv.threshold(gim, thr, 255, cv.THRESH_BINARY)



In [ ]:
#skimage multi threshold

import numpy as np
from skimage.filters import threshold_multiotsu


def multi_thresh(img, t):
    im = img.copy()
    s = im.shape
    a = t[0]
    b = t[1]
    for i in range(s[0]):
        for j in range(s[1]):
            if (im[i, j] < a or im[i, j] > b):
                im[i, j] = 0
            else:
                im[i, j] = 255
    return im


thresholds = threshold_multiotsu(gim)

result = multi_thresh(gim, thresholds)


In [ ]:
# multi thr implementation

img = gim
a = 0
b = 255
n = 4  # number of thresholds (better choose even value)
k = 0.7  # free variable to take any positive value
T = []  # list which will contain 'n' thresholds


def multiThresh(img, a, b):
    if a > b:
        s = -1
        m = -1
        return m, s

    img = np.array(img)
    t1 = (img >= a)
    t2 = (img <= b)
    X = np.multiply(t1, t2)
    Y = np.multiply(img, X)
    s = np.sum(X)
    m = np.sum(Y)/s
    return m, s


for i in range(int(n/2-1)):
    img = np.array(img)
    t1 = (img >= a)
    t2 = (img <= b)
    X = np.multiply(t1, t2)
    Y = np.multiply(img, X)
    mu = np.sum(Y)/np.sum(X)

    Z = Y - mu
    Z = np.multiply(Z, X)
    W = np.multiply(Z, Z)
    sigma = math.sqrt(np.sum(W)/np.sum(X))

    T1 = mu - k*sigma
    T2 = mu + k*sigma

    x, y = multiThresh(img, a, T1)
    w, z = multiThresh(img, T2, b)

    T.append(x)
    T.append(w)

    a = T1+1
    b = T2-1
    k = k*(i+1)

T1 = mu
T2 = mu+1
x, y = multiThresh(img, a, T1)
w, z = multiThresh(img, T2, b)
T.append(x)
T.append(w)
T.sort()
result= multi_thresh(gim, T)

adaptive local threshold binarization

In [ ]:
# cv2 MEAN_C

mean_c = cv.adaptiveThreshold(gim, 255, cv.ADAPTIVE_THRESH_MEAN_C,
                                cv.THRESH_BINARY, 199, 5)

In [ ]:
# cv2 GAUSSIAN_C

gaussian_c = cv.adaptiveThreshold(gim, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C,
                                cv.THRESH_BINARY, 199, 5)



In [ ]:
#the implementation of the rest

import enum
import math


class Threshold(enum.Enum):
    def default(subImg=[]):
        return 127
    
    def niBlack(subImg=[], k=0.06):
        m = np.mean(subImg)
        std = np.std(subImg)
        return m - k*std

    def sauvola(subImg=[], k=0.06, R=128):
        m = np.mean(subImg)
        std = np.std(subImg)
        return m * (1 + k * (std/R - 1))

    def brenser(subImg=[], k=0.06, R=128):
        imax = np.amax(subImg)
        imin = np.amin(subImg)
        return 0.5*(imin + imax)

    def nick(subImg=[], k=0.06, R=128):
        m = np.mean(subImg)
        n = subImg.shape[0] * subImg.shape[1]
        sum = np.sum(subImg**2)
        return m + k*(math.sqrt((sum - m)/n))

    def wolf(subImg=[], R=0, M=0, a=0.5):
        m = np.mean(subImg)
        std = np.std(subImg)
        return ((1-a)*m + a*M + ((a*std)/R)*(m-M))


# masking the image and applying the thr methodes to the box
def adaptiveThreshold(im, sizeI=4, sizeJ=4, k=1.0, type=Threshold.default):
    img = im.copy()

    s1 = img.shape[0]
    s2 = img.shape[1]

    sizeI = int(sizeI/2)
    sizeJ = int(sizeJ/2)

    if(sizeI <= 0):
        sizeI = 1

    if(sizeJ <= 0):
        sizeJ = 1
    
    if (type == Threshold.wolf):
        R = -1
        M = im[0, 0]
        for i in range(0, s1):
            for j in range(0, s2):

                posI1 = i-sizeI
                posI2 = i+sizeI

                posJ1 = j-sizeJ
                posJ2 = j+sizeJ

                if(posI1 < 0):
                    posI1 = 0

                if(posI2 >= s1):
                    posI2 = s1 - 1

                if(posJ1 < 0):
                    posJ1 = 0

                if(posJ2 >= s2):
                    posJ2 = s2 - 1

                subImg = im[posI1:posI2, posJ1:posJ2]
                std = np.std(subImg)
                if (std > R):
                    R = std
                if (im[i, j] < M):
                    M = im[i, j]

    for i in range(0, s1):
        for j in range(0, s2):

            posI1 = i-sizeI
            posI2 = i+sizeI

            posJ1 = j-sizeJ
            posJ2 = j+sizeJ

            if(posI1 < 0):
                posI1 = 0

            if(posI2 >= s1):
                posI2 = s1 - 1

            if(posJ1 < 0):
                posJ1 = 0

            if(posJ2 >= s2):
                posJ2 = s2 - 1

            subImg = im[posI1:posI2, posJ1:posJ2]

            if (type == Threshold.wolf):
                img[i, j] = type(subImg, R, M)
            else:
                img[i, j] = type(subImg)

    t = img

    final_img = im.copy()
    final_img[final_img <= t] = 0
    final_img[final_img > t] = 255

    final_img[:, 0:9] = 0
    final_img[:, -9:] = 0
    final_img[0:9, :] = 0
    final_img[-9:, :] = 0

    return final_img


result = adaptiveThreshold(gim, 15, 15, 0.2, Threshold.niBlack)

skimage: threshold_niblack, threshold_sauvola, threshold_otsu

In [ ]:
from skimage import io
from skimage.filters import (
    threshold_niblack, threshold_sauvola, threshold_otsu)


input_image = '/home/jamal/Documents/internship/images/image.png'


image_color = io.imread(input_image)

image_gray = io.imread(input_image, as_gray=True)


skimage sauvola

In [ ]:
window_size = 15
thresh = threshold_sauvola(image_gray, window_size=window_size, k=0.2)

thresh = image_gray > thresh

# io.imshow(thresh)
# io.show()
io.imsave('/home/jamal/Documents/internship/images/scikit_sauvola.png', thresh)


skimage niblack

In [ ]:
window_size = 15
thresh = threshold_niblack(image_gray, window_size=window_size, k=0.2)

thresh = image_gray > thresh

# io.imshow(thresh)
# io.show()
io.imsave('/home/jamal/Documents/internship/images/scikit_niblack.png', thresh)


skimage otsu

In [ ]:
window_size = 15
thresh = threshold_otsu(image_gray, window_size)

thresh = image_gray > thresh

# io.imshow(thresh)
# io.show()
io.imsave('/home/jamal/Documents/internship/images/scikit_otsu.png', thresh)
